# Import Libraries

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from torch import tensor
from sklearn.model_selection import train_test_split
import pandas as pd
from PIL import Image
import os

# Load and Prepare the Dataset

### Load the CSV files

In [4]:
# Define the root directory of the dataset
dataset_root = '/kaggle/input/ai-vs-human-generated-dataset/'
# Load the train CSV file
train_df = pd.read_csv(os.path.join(dataset_root, 'train.csv'), index_col=0)
# Load the test CSV file
test_df = pd.read_csv(os.path.join(dataset_root, 'test.csv'))

train_df.head()

,file_name,label
0,train_data/a6dcb93f596a43249135678dfcfc17ea.jpg,1
1,train_data/041be3153810433ab146bc97d5af505c.jpg,0
2,train_data/615df26ce9494e5db2f70e57ce7a3a4f.jpg,1
3,train_data/8542fe161d9147be8e835e50c0de39cd.jpg,0
4,train_data/5d81fa12bc3b4cea8c94a6700a477cf2.jpg,1


In [5]:
# Split into training and validation (80% train, 20% validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['label'])

print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}")

Train size: 63960, Validation size: 15990


### Define a custom Dataset classes

In [6]:
class ImageDataset(Dataset):
    def __init__(self, df, root_dir, transform=None, is_test=False):
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
        self.is_test = is_test  # Flag to indicate if this is the test dataset

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if self.is_test:
            # Use the first column (assumed to be 'id' or 'file_name')
            img_path = os.path.join(self.root_dir, self.df.iloc[idx, 0])  
        else:
            # Use column names instead of hardcoded index
            img_path = os.path.join(self.root_dir, self.df['file_name'].iloc[idx])  
            label = int(self.df['label'].iloc[idx])  

        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        if self.is_test:
            return image, -1 
        else:
            return image, label

### Define transformations

In [7]:
# Training Transform (with data augmentation)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.RandomHorizontalFlip(),  # Augmentation: Randomly flip images
    # transforms.RandomRotation(10),      # Augmentation: Rotate images slightly
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Test/Validation Transform (NO augmentation)
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Keep it consistent
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

### Create datasets and dataloaders

In [8]:
# Create datasets
train_dataset = ImageDataset(train_df, dataset_root, transform=train_transform, is_test=False)
val_dataset = ImageDataset(val_df, dataset_root, transform=test_transform, is_test=False)
test_dataset = ImageDataset(test_df, dataset_root, transform=test_transform, is_test=True)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Print dataset sizes
print(f'Train dataset size: {len(train_dataset)}')
print(f'Validation dataset size: {len(val_dataset)}')
print(f'Test dataset size: {len(test_dataset)}')

Train dataset size: 63960
Validation dataset size: 15990
Test dataset size: 5540


# Define the Model

##### Load a pre-trained model (e.g., ResNet50) and modify the final layer

In [9]:
from torchvision.models import resnet50, ResNet50_Weights

# Load ResNet50 model with pre-trained weights
model = resnet50(weights=ResNet50_Weights.DEFAULT)

# Modify the fully connected (fc) layer for binary classification
model.fc = nn.Linear(model.fc.in_features, 2)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 224MB/s]


In [10]:
for param in model.parameters():
    param.requires_grad = False  # Freeze all layers

for param in model.layer4.parameters():  # Unfreeze the last ResNet block
    param.requires_grad = True  
for param in model.fc.parameters():
    param.requires_grad = True  # Keep final layer trainable

### Move the model to the GPU if available

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define Loss Function and Optimizer

In [12]:
from torch.optim.lr_scheduler import StepLR
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-4)
#AdamW helps improve generalization.
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Reduce LR by 10x every 5 epochs


# Train the Model

In [13]:
num_epochs = 20  # Increased to give early stopping a chance to work
patience = 3  # Number of epochs to wait before stopping if no improvement
best_val_acc = 0  # Track the best validation accuracy
counter = 0  # Counter for patience

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training loop
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)
    
    train_accuracy = 100 * correct_train / total_train
    avg_loss = running_loss / len(train_loader)

    # Validation loop
    model.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    val_accuracy = 100 * correct_val / total_val

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%')
    print(f'Learning Rate: {scheduler.get_last_lr()[0]}')  # Print current learning rate
    
    # Update learning rate
    scheduler.step()
    
    # Early Stopping Logic
    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        counter = 0  # Reset patience counter
        torch.save(model.state_dict(), 'best_model.pth')  # Save best model
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            break  # Stop training if no improvement for `patience` epochs

Epoch 1/20, Loss: 0.0573, Training Accuracy: 97.81%, Validation Accuracy: 99.27%
Learning Rate: 0.0005
Epoch 2/20, Loss: 0.0129, Training Accuracy: 99.57%, Validation Accuracy: 99.04%
Learning Rate: 0.0005
Epoch 3/20, Loss: 0.0076, Training Accuracy: 99.75%, Validation Accuracy: 99.29%
Learning Rate: 0.0005
Epoch 4/20, Loss: 0.0060, Training Accuracy: 99.80%, Validation Accuracy: 99.36%
Learning Rate: 0.0005
Epoch 5/20, Loss: 0.0043, Training Accuracy: 99.86%, Validation Accuracy: 99.45%
Learning Rate: 0.0005
Epoch 6/20, Loss: 0.0020, Training Accuracy: 99.94%, Validation Accuracy: 99.52%
Learning Rate: 5e-05
Epoch 7/20, Loss: 0.0008, Training Accuracy: 99.98%, Validation Accuracy: 99.57%
Learning Rate: 5e-05
Epoch 8/20, Loss: 0.0004, Training Accuracy: 100.00%, Validation Accuracy: 99.59%
Learning Rate: 5e-05
Epoch 9/20, Loss: 0.0003, Training Accuracy: 99.99%, Validation Accuracy: 99.57%
Learning Rate: 5e-05
Epoch 10/20, Loss: 0.0002, Training Accuracy: 100.00%, Validation Accuracy: 

# Evaluate the Model

In [ ]:
# Make predictions on test set
model.eval()
predictions = []

with torch.no_grad():
    for images, _ in test_loader:  # No labels in the test set
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

# Ensure IDs are correctly extracted from test_df
submission_df = pd.DataFrame({'id': test_df.iloc[:, 0], 'label': predictions})

# Save predictions to CSV
submission_df.to_csv('submission.csv', index=False)

# Check the first few rows
print(submission_df.head())

                                                  id  label
0  test_data_v2/1a2d9fd3e21b4266aea1f66b30aed157.jpg      1
1  test_data_v2/ab5df8f441fe4fbf9dc9c6baae699dc7.jpg      0
2  test_data_v2/eb364dd2dfe34feda0e52466b7ce7956.jpg      0
3  test_data_v2/f76c2580e9644d85a741a42c6f6b39c0.jpg      0
4  test_data_v2/a16495c578b7494683805484ca27cf9f.jpg      0


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Collect predictions and labels
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:  # Use val_loader or test_loader
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute metrics
precision = precision_score(all_labels, all_preds, average='binary')
recall = recall_score(all_labels, all_preds, average='binary')
f1 = f1_score(all_labels, all_preds, average='binary')

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Precision: 0.9956
Recall: 0.9957
F1 Score: 0.9957
